In [43]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

In [44]:
api_key = input("Please enter your YouTube Data API key: ")
channel_id='UCtAXJ4DNpshfVjLIMq9pIRw'
youtube= build('youtube','v3',developerKey=api_key)

In [45]:
def get_channel(youtube, channel_id):
    all_data = []
    request = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=channel_id
    )
    response = request.execute() 
    
    # Check if 'items' key exists in the response
    if 'items' in response:
        for i in range(len(response['items'])):
            data = dict(
                channelName=response['items'][i]['snippet']['title'],
                subscribers=response['items'][i]['statistics']['subscriberCount'],
                views=response['items'][i]['statistics']['viewCount'],
                totalVideos=response['items'][i]['statistics']['videoCount'],
                playlistId=response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
            )
            all_data.append(data)
    else:
        print("No 'items' found in response.")
    
    return pd.DataFrame(all_data)



In [46]:
channel_stats=get_channel(youtube,channel_id)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Rohit & Kanupriya,2290000,2191409562,330,UUtAXJ4DNpshfVjLIMq9pIRw


In [47]:
playlistid=channel_stats['playlistId']
playlistid

0    UUtAXJ4DNpshfVjLIMq9pIRw
Name: playlistId, dtype: object

In [48]:
def get_video_ids(youtube, playlist_id,video_ids):

    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [49]:
video_ids = []
for i in range(len(playlistid)):
    playlist_id=playlistid[i]
    Video_ids=get_video_ids(youtube, playlist_id,video_ids)
    
len(Video_ids)

330

In [50]:
def get_comments_in_videos(youtube, video_ids):
    all_data = []

    for video_id in video_ids:
        try:
            # Retrieve video title and statistics
            video_request = youtube.videos().list(
                part="snippet,statistics",
                id=video_id
            )
            video_response = video_request.execute()
            video_title = video_response['items'][0]['snippet']['title']
            video_likes = video_response['items'][0]['statistics']['likeCount']
            video_views = video_response['items'][0]['statistics']['viewCount']

            # Retrieve comments for the video
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id,
            )
            response = request.execute()

            for comment in response['items']:
                topLevelComment = comment['snippet']['topLevelComment']['snippet']
                data = {
                    'videoId': video_id,
                    'title': video_title,
                    'videoLikes': video_likes,
                    'videoViews': video_views,
                    'commenter_name': topLevelComment['authorDisplayName'],
                    'comments': topLevelComment['textOriginal'],
                }
                all_data.append(data)

                # Add replies if any
                if 'replies' in comment and 'comments' in comment['replies']:
                    for reply in comment['replies']['comments']:
                        reply_data = {
                            'videoId': video_id,
                            'title': video_title,
                            'videoLikes': video_likes,
                            'videoViews': video_views,
                            'commenter_name': reply['snippet']['authorDisplayName'],
                            'comments': reply['snippet']['textOriginal'],
                        }
                        all_data.append(reply_data)

        except Exception as e:
            # When an error occurs - most likely because comments are disabled on a video
            print(f'Could not get data for video {video_id}: {e}')

    return pd.DataFrame(all_data)


In [51]:
comments = get_comments_in_videos(youtube, video_ids)
comments

,videoId,title,videoLikes,videoViews,commenter_name,comments
0,Fj-g-cmSzbo,Beautiful but toxic. #Funny #comedy #couple #e...,56482,808475,@RohitandKanu,You guys know of any other beautiful but toxic...
1,Fj-g-cmSzbo,Beautiful but toxic. #Funny #comedy #couple #e...,56482,808475,@rignudignu7200,A Porsche
2,Fj-g-cmSzbo,Beautiful but toxic. #Funny #comedy #couple #e...,56482,808475,@mastmogachi,Monkey shoulder 🍾 is looking very beautiful ri...
3,Fj-g-cmSzbo,Beautiful but toxic. #Funny #comedy #couple #e...,56482,808475,@DD-pe3no,😍😅👌🏻
4,Fj-g-cmSzbo,Beautiful but toxic. #Funny #comedy #couple #e...,56482,808475,@MrKamei,my crush
...,...,...,...,...,...,...
7493,xxwNglca0Bo,Limca Book of World Records - Renu Sukheja,1073,149466,@dmcindianfan5374,Just went back 10 years ago 😂😅
7494,xxwNglca0Bo,Limca Book of World Records - Renu Sukheja,1073,149466,@coolbreeze1262,Is the record still to her ? ❤
7495,xxwNglca0Bo,Limca Book of World Records - Renu Sukheja,1073,149466,@Notebook-ur1st,Wow!! She looks different 😳 \nI follow them fo...
7496,xxwNglca0Bo,Limca Book of World Records - Renu Sukheja,1073,149466,@Prathikraj_mysorean,one can see Rohit's Journey 😂


In [52]:
comments.to_csv('Data\\commentsData.csv', index=False)